### <span style="color:red">IMPORTANT: Only modify cells which have the following comment:</span>
```python
# Modify this cell
```
##### <span style="color:red">Do not add any new cells when you submit the homework. Only use the already provided python packages.</span>

## Analysis of stock prices using PCA / Notebook 1

In this take-home final you are to analyze the daily changes in stock prices using PCA and to mesure the dimension of stock sequences.

We Start by downloading data and pre-processing it to make it ready for analysis using Spark.

In [ ]:
import sys,os
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
%matplotlib inline

from time import time
import math
import pandas as pd
from glob import glob
import pickle


### Stock Info
The Pickle file `Tickers.pkl` contains information about the stocks in the SP500.

* `TickerInfo` - a pandas table that stores the name, sector, and sector ID for 505 stocks
* `Tickers` - A list of the stocks that we are going to analyze (each student should eliminate a few of these stocks before doing their analysis)


In [ ]:
D=pickle.load(open('Tickers.pkl','rb'))
TickerInfo=D['TickerInfo']
Tickers=D['Tickers']
Sectors={'Consumer Discretionary':'CD',
 'Consumer Staples':'CS',
 'Energy':'EN',
 'Financials':'FIN',
 'Health Care':'HC',
 'Industrials':'INDS',
 'Information Technology':'IT',
 'Materials':'MAT',
 'Real Estate':'RE',
 'Telecommunication Services':'TS',
 'Utilities':'UTIL'}

In [ ]:
TickerInfo.head()

### Differences between the stocks lists
In the following you will work with three different sets of stocks:

1. The stocks listed in `TickerInfo`
2. The stocks listed in `Tickers`
3. The stock files that you will download from S3.

* The stocks you will analyze are those in `Tickers` less the one that you were instructed to remove.
* The Files you will download contain all of the stocks in `Tickers` plus a few stocks that are skipped because they are outliers.
* The Stocks in `TickerInfo` include most of the stocks in `Tickers`, but there are a few missing. When we will represent each stock with its `SECTOR_ID` these stocks will be represented by their Ticker.

In [ ]:
len(Tickers),len(TickerInfo)

### Stock and sector information
`TickerInfo` is a pandas table containing, for each Ticker, the company name, the sector, and a sector ID. There are 11 sectors. Some, such as `Consumer Discretionary` and `Information Technology` include many stocks while others, such as `Telecommunication Services` include very few.

In [ ]:
from collections import Counter
L=Counter(TickerInfo['Sector']).items()
print 'Sector ID\t\tSector Name\tNo. of Stocks'
print '=========\t\t===========\t============='
for l in L:
    print '%s\t%30s\t%d'%(Sectors[l[0]],l[0],l[1])

### Download Data
The data is a directory with .csv files, one for each stock. This directory has been tarred and uploaded to 
S3, at:

https://mas-dse-open.s3.amazonaws.com/spdata_csv.tgz

**Download and untar** the file. This creates a directory called **spdata_csv**

## Read Data and create a single table

Your task in this notebook is to read the stock-information `.csv` files, extract from them the column 
`Adj. Open` and combine them into a single `.csv` file containing all of the information that is relevant for later analysis.

Below we suggest a particular sequence of steps, you can either follow these steps, or do this in your own way.  The end result should be a file called `SP500.csv` which stores the information described below.

### Convert files into pandas dataframes

In this step you read all of the relevant information into a large dictionary we call `Tables`.

The key to this dictionary is the stocks "ticker" which corresponds to the file name excluding the `.csv` extension.

You should read in all of the files in the directory `spdata_csv` other than:

* Files for tickers that are not in the list `Tickers`.
* Files for tickers that were listed in the email you got for this final.

In the email you got about this final, there is a list of tickers that you should omit from your analysis.

# Task 1:
Create a function **getTables** which returns the dictionary *Tables* as described above. 

######  <span style="color:blue">Code:</span>
```python
Tables = getTables()
print type(Tables)
print len(Tables)
Tables['IBM'].head()
```

######  <span style="color:magenta">Output:</span>
```python
dict
476

```
<p><img alt="" src="figs/IBM.jpg"style="height:180px"  /></p>


In [ ]:
def getTables(Data_dir="directory/path/to/spdata_csv")
    #Data_dir: directory path to spdata_csv , default path should be your local path to the file 
    
    Tables={}
    # ... write implementation
    
    return Tables   #<-- a dictionary as described above



In [ ]:
from Tester import prepare as tasks
Tables = tasks.ex1(getTables)

### Computing diffs and combining into a single table

The next step is to extract from each table the relevant prices, compute an additional quantity we call `diff` and create a single combined pandas dataframe.

The price we ue is the **Adjusted Open Price** which is the price when the stock exchange opens in the morning. We use the **adjusted** price which eliminates technical adjustments such as stock splits.

It is more meaningful to predict *changes* in prices than prices themselves. We therefor compute for each stock a `Diffs` sequence in which $d(t)=\log \frac{p(t+1)}{p(t)}$ where $p(t)$ is the price at time $t$ and $d(t)$ is the price diff or the price ratio.

Obviously, if we have a price sequence of length $T$ then the length of the diff sequence will be $T-1$. To make the price sequence and the diff sequence have the same length we eliminate the last day price for each sequence.

Your task in this step is to join the stock tables by date, compute the diff seqeunce, and create one large Pandas DataFrame where the row index is the date, and there are two columns for each ticker. For example for the ticker `IBM`, there would be two columns `IBM_P` and `IBM_D`. The first corresponds to the prices of the IBM stock $p(t)$ and the second to the price difference $d(t)$

# Task 2: 

Create a function **computeDiffs**, which creates the dataframe Diffs as specified above


######  <span style="color:blue">Code:</span>
```python
Diffs = computeDiffs(Tables)
print type(Diffs)
print Diffs.shape[1]
print "IBM_D" in Diffs.columns and "IBM_P" in Diffs.columns

```

######  <span style="color:magenta">Output:</span>
```python
<class 'pandas.core.frame.DataFrame'>
952
True
```


In [ ]:
def computeDiffs(Tables):
    #Tables: the dictionary Tables that was computed in Task1 
    
    Diffs=pd.DataFrame()
    # ... write implementation
    
    return Diffs  #<-- a dataframe as described above


In [ ]:
from Tester import prepare as tasks
Diffs = tasks.ex2( computeDiffs(Tables) )

In [ ]:
# plot some stocks
Diffs[['AAPL_P','MSFT_P','IBM_P']].plot(figsize=(14,10));
plt.grid()

Lastly,we now save *Diffs* as a csv

In [ ]:
Diffs.to_csv('SP500.csv')